# LoRA 模型訓練
1. 執行階段->變更執行階段類型->硬體加速器選擇GPU
2. 掛接雲端硬碟
3. 將資料集放在/content/drive/MyDrive/LoRA_training/datasets
4. 建立環境
5. 訓練參數設置
6. 開始訓練LoRA

詳細步驟請參考[巴哈教學](https://home.gamer.com.tw/artwork.php?sn=5652683)

In [ ]:
#@title 掛接雲端硬碟
from google.colab import drive
drive.mount('/content/drive')
!cd /content/drive/MyDrive/
!mkdir /content/drive/MyDrive/LoRA_training/
!mkdir /content/drive/MyDrive/LoRA_training/datasets
!mkdir /content/drive/MyDrive/LoRA_training/output
!mkdir /content/drive/MyDrive/LoRA_training/log

In [ ]:
#@title 建立環境
!sudo apt-get update -y && sudo apt-get install python3.10 python3.10-dev python3.10-distutils libpython3.10-dev python3.10-venv

# Link python3 to 3.10.
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2

# Install pip.
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.9

# Install your dependencies. Pin IPython and traitlets versions, as traitlets version 5.8 contains a breaking change that prevents google-colab server extension from working properly.
# Some of these packages may not be required.
!python3 -m pip install ipython==7.9.0 traitlets==5.7.1 jupyter psutil setuptools ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# Copy the already installed `google-colab` package over. It depends on an _old version_ of pandas, so building the wheel won't work.
!ln -s /usr/local/lib/python3.8/dist-packages/google \
       /usr/local/lib/python3.10/dist-packages/google

# Reinstall jupyter server extension.
!jupyter serverextension list
!jupyter notebook --config=/usr/local/etc/jupyter/jupyter_notebook_config.json

#Reinstall pip
!python3 -m ensurepip --upgrade

%cd /content
!git clone https://github.com/kohya-ss/sd-scripts
!git checkout 53d60543e59d6fdf3a6b5d0d15019487d2e415db
%cd /content/sd-scripts

!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install --upgrade -r requirements.txt
!pip install -U -I --no-deps https://github.com/C43H66N12O12S2/stable-diffusion-webui/releases/download/linux/xformers-0.0.14.dev0-cp310-cp310-linux_x86_64.whl


In [ ]:
#@title 下載 anything-v3-fp32-pruned.safetensors { display-mode: "form" }
import os
from google.colab import drive

%cd /content/drive/MyDrive/Stable_Diffusion_Webui/Stable-diffusion

if not os.path.exists('anything-v3-fp32-pruned.safetensors'):
  print("下載SD模型...") 
  !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp32-pruned.safetensors

In [ ]:
#@title 訓練參數設置

#@markdown ##專案名稱
project_name = "kacya07" #@param {type:"string"}
train_data_dir = "/content/drive/MyDrive/LoRA_training/datasets/" + project_name
output_dir = "/content/drive/MyDrive/LoRA_training/output/" + project_name
logging_dir = "/content/drive/MyDrive/LoRA_training/log"
#@markdown 將資料集資料夾放在 `/content/drive/MyDrive/LoRA_training/datasets` ，資料夾名稱為 `project_name`。

#@markdown ##訓練相關參數
#@markdown Base Model路徑
model_path = "/content/drive/MyDrive/Stable_Diffusion_Webui/Stable-diffusion/anything-v3-fp32-pruned.safetensors" #@param {type:"string"}
#@markdown 圖片解析度
width = 576 #@param {type:"slider", min:256, max:1024, step:64}
height = 1024 #@param {type:"slider", min:256, max:1024, step:64}
#@markdown batch大小
train_batch_size = 1 #@param {type:"slider", min:1, max:8, step:1}
#@markdown 最大訓練epochs
max_train_epochs = 30 #@param {type:"integer"}
#@markdown 每n個epochs存檔一次
save_every_n_epochs = 5 #@param {type:"integer"}
#@markdown LoRA模型大小
network_dim = 64 #@param {type:"slider", min:1, max:128, step:1}
#@markdown 打亂標籤
use_shuffle_caption = True #@param {type:"boolean"}

#@markdown ##學習率
lr = 1e-4 #@param {type:"number"}
unet_lr = 1e-4 #@param {type:"number"}
text_encoder_lr = 1e-5 #@param {type:"number"}
lr_scheduler = "cosine_with_restarts" #@param ["constant", "constant_with_warmup", "cosine", "cosine_with_restarts", "linear", "polynomial"]
#@markdown 只有在 `lr_scheduler` 為constant_with_warmup時需要填寫 `lr_warmup_steps` 。
lr_warmup_steps = 0 #@param {type:"integer"}

#@markdown ##輸出模型
save_model_as = "safetensors" #@param ["safetensors", "ckpt", "pt"]

if (use_shuffle_caption):
  shuffle_caption = "--shuffle_caption"
else:
  shuffle_caption = ""
  

In [ ]:
!nvidia-smi

In [ ]:
#@title 開始訓練LoRA
%cd /content/sd-scripts
!accelerate launch \
  --mixed_precision 'fp16' \
  --num_cpu_threads_per_process=2 \
  "train_network.py" \
  --cache_latents \
  --enable_bucket \
  --use_8bit_adam \
  --mem_eff_attn \
  --pretrained_model_name_or_path=$model_path \
  --train_data_dir=$train_data_dir \
  --resolution=$width,$height \
  --output_dir=$output_dir \
  --train_batch_size=$train_batch_size \
  --lr_scheduler=$lr_scheduler \
  --lr_warmup_steps=$lr_warmup_steps \
  --max_train_epochs=$max_train_epochs \
  --use_8bit_adam \
  --mixed_precision=fp16 \
  --save_every_n_epochs=$save_every_n_epochs \
  --seed=4242 \
  --save_precision=fp16 \
  --logging_dir=$logging_dir \
  --save_model_as=$save_model_as \
  --network_module=networks.lora \
  --text_encoder_lr=$text_encoder_lr \
  --unet_lr=$unet_lr \
  --learning_rate=$lr \
  --network_dim=$network_dim \
  --network_alpha=$network_dim \
  --clip_skip=2 \
  --caption_extension=".txt" \
  --output_name=$project_name \
  $shuffle_caption